# ANN2DL Homework 1: Face Mask Detection
## Splitting training and validation pictures in Keras-compatible directory structures
Validation set size is chosen so as to be equal to the test set size given.

In [1]:
# Cell output set upf for Jupyter
from pathlib import Path
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import json
import random

SEED = 1234
random.seed(SEED)

test_pictures_n = 450 # validation set size is set equal to test set size
target_file_name = "train_gt.json"
dataset_name = "MaskDataset"

# Setting up directory structure
Path().joinpath(dataset_name, "validation").mkdir(parents=True, exist_ok=True)
Path().joinpath(dataset_name, "training", "NO_PERSON").mkdir(parents=True, exist_ok=True)
Path().joinpath(dataset_name, "training", "ALL_THE_PEOPLE").mkdir(parents=True, exist_ok=True)
Path().joinpath(dataset_name, "training", "SOMEONE").mkdir(parents=True, exist_ok=True)
Path().joinpath(dataset_name, "validation", "NO_PERSON").mkdir(parents=True, exist_ok=True)
Path().joinpath(dataset_name, "validation", "ALL_THE_PEOPLE").mkdir(parents=True, exist_ok=True)
Path().joinpath(dataset_name, "validation", "SOMEONE").mkdir(parents=True, exist_ok=True)

# Files are moved from the training directory to the corresponding folders
# both for training and for validation
with open(str(Path().joinpath(dataset_name, target_file_name))) as f:
    data = json.load(f)
    pictures = list(data.keys())
    random.shuffle(pictures)
    validation_pictures = pictures[0:test_pictures_n]
    for path in Path().joinpath(dataset_name, "training").glob("*.jpg"):
        if path.name in validation_pictures:
            file_destination = str(Path().joinpath(dataset_name, "validation", path.name))
            path.rename(file_destination)
            path = Path(file_destination)
        if data[path.name] == 0:
            path.rename(str(Path(path.parent).joinpath("NO_PERSON", path.name)))
        elif data[path.name] == 1:
            path.rename(str(Path(path.parent).joinpath("ALL_THE_PEOPLE", path.name)))
        elif data[path.name] == 2:
            path.rename(str(Path(path.parent).joinpath("SOMEONE", path.name)))
        else:
            raise ValueError("Unrecognized label in " + target_file_name + " allowed values are 0, 1, 2 found: " + str(data[path.name]))
            

## Dataset handling and augmentation
ImageDataGenerator handles data augmentation and the data flows from the directory which is now compliant with Keras requirements

In [3]:
from PIL import Image
import numpy as np

# Getting the size of all images
image_size_set = set()
for path in Path().joinpath(dataset_name).glob("**/*.jpg"):
    image_size_set.add(Image.open(str(path)).size)
    
# Looking for smaller width and height in the dataset
w_min = np.inf
h_min = np.inf
for t in image_size_set:
    if t[0] < w_min:
        w_min = t[0]
    if t[1] < h_min:
        h_min = t[1]
print("Images must be resized at least to: " + str(w_min) + " * " + str(h_min))

Images must be resized at least to: 345 * 256


In [4]:
import tensorflow as tf
import numpy as np

tf.random.set_seed(SEED)

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Batches
bs = 32

# Data augmentation switch
apply_data_augmentation = False
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(zoom_range=0.2,
                                        rescale=1./255)
else:
       train_data_gen = ImageDataGenerator(rescale=1./255)
valid_data_gen = ImageDataGenerator(rescale=1./255)

In [6]:
class CustomAugment(object):
    def __call__(self, image):        
        img = self._random_apply(self._color_drop, image, p=0.8)
        return img
    def __name__(self):
        return "CustomAugment"

    def _color_drop(self, x):
        image = tf.image.rgb_to_grayscale(x)
        image = tf.tile(x, [1, 1, 1, 3])
        return x
    
    def _random_apply(self, func, x, p):
        return tf.cond(
          tf.less(tf.random.uniform([], minval=0, maxval=1, dtype=tf.float32),
                  tf.cast(p, tf.float32)),
          lambda: func(x),
          lambda: x)

In [7]:
dataset_dir = Path().joinpath(dataset_name)


# Target Image Shape
# Largest image size would be 358 x 256
img_h = 256
img_w = 256

num_classes = 3
classes = ["NO_PERSON",
          "ALL_THE_PEOPLE",
          "SOMEONE"]

training_dir = dataset_dir.joinpath("training")
train_gen = train_data_gen.flow_from_directory(str(training_dir),
                                              batch_size=bs,
                                              classes=classes,
                                              class_mode="categorical",
                                              shuffle=True,
                                              target_size=(img_h, img_w),
                                              seed=SEED)

validation_dir = dataset_dir.joinpath("validation")
valid_gen = valid_data_gen.flow_from_directory(str(validation_dir),
                                              batch_size=bs,
                                              classes=classes,
                                              class_mode="categorical",
                                              shuffle=True,
                                              target_size=(img_h, img_w),
                                              seed=SEED)

Found 5164 images belonging to 3 classes.
Found 450 images belonging to 3 classes.


In [8]:
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                              output_types=(tf.float32, tf.float32),
                                              output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
train_dataset = train_dataset.repeat()

valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                              output_types=(tf.float32, tf.float32),
                                              output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
valid_dataset = valid_dataset.repeat()

## Model specification - 1
The approach in https://arxiv.org/pdf/2009.08369.pdf is used as a starting point so:
- Inception V3 Model stripped of the last layer
- 5 layers added:
    - average pooling layer 5x5
    - flattening layer
    - dense layer with 128 Neurons with ReLU activation
    - dropout layer with 0.5 dropout probability
    - decision layer with softmax activations
- we generally leverage transfer learning but being that the last pooling layer is brand new we are tuning its parameters and the ones of the multi-layer perceptron

- Random gray scale, instead of uniform gray scaling in the paper, is applied together with flips and zooms which in the paper are not present

Batch Size = 8 -> 32  


TODO: retrain it with pre-processed dataset
TODO: evaluate unfreezing some inception layers  
TODO: evaluate using full inception and not adding the new layer  
TODO: evaluate adding more convolutional layers after inception  

## Model Specification - 2
Based on: https://gist.github.com/didacroyo/839bd1dbb67463df8ba8fb14eb3fde0c
Transfer Learning with Inception V3.
1. Train all top layers and freeze Inception V3 for 20 epochs without early stopping
2. Keep layers between 0-249 frozen and unfreeze the rest, lower the learning rate and re-train for 100 epochs with early stopping  
Batch Size = 8 -> 4
epochs = 20 + 100 -> 100 (skipped first step)
Early stopping on second training  
Data Augmentation Standard + Custom Grey scaling

## Model Specification - 3
Vgg16 with pre-processing on input  
batch size = 16  
Early Stopping = True  
Data Augmentation Custom (Data Augmentation Switch to False)
TODO: try with VGG19 (is slow)

## Model Specification - 4 (ShuffleNet)


# Model Specification - 5

In [13]:
# building a linear stack of layers with the sequential model
model = tf.keras.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=(img_h, img_w, 3)))

# convolutional layer
model.add(tf.keras.layers.Conv2D(25, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))

model.add(tf.keras.layers.Conv2D(25, (5, 5), activation='relu', strides=(1, 1), padding='same'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='same'))

model.add(tf.keras.layers.Conv2D(50, (5, 5), activation='relu', strides=(2, 2), padding='same'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='same'))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(70, (3, 3), activation='relu', strides=(2, 2), padding='same'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='valid'))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=100, activation='relu'))
model.add(tf.keras.layers.Dense(units=100, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')


## Callbacks

In [14]:
from datetime import datetime

callbacks_dir = Path().joinpath("Callbacks")
callbacks_dir.mkdir(parents=True, exist_ok=True)

now = datetime.now().strftime("%b%d_%H-%M-%S")

model_name = "CNN"

callback_dir = callbacks_dir.joinpath(model_name + '_' + str(now))
callback_dir.mkdir(parents=True, exist_ok=True)

callbacks = []

# Model checkpoint
ckpt_dir = callback_dir.joinpath("ckpts")
ckpt_dir.mkdir(parents=True, exist_ok=True)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=str(ckpt_dir.joinpath("cp.ckpt")), 
                                                   save_weights_only=True)
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
tb_dir = callback_dir.joinpath("tb_logs")
tb_dir.mkdir(parents=True, exist_ok=True)
    
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=str(tb_dir),
                                             profile_batch=0,
                                             histogram_freq=1) 
callbacks.append(tb_callback)

# Early Stopping
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)

# Model Fitting

In [15]:
model.fit(x=train_dataset,
          epochs=10,
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks)

Epoch 1/10
162/162 [==============================] - 94s 578ms/step - loss: 1.2070 - accuracy: 0.3734 - val_loss: 1.2307 - val_accuracy: 0.3622
Epoch 2/10
162/162 [==============================] - 83s 512ms/step - loss: 1.0698 - accuracy: 0.4355 - val_loss: 1.0524 - val_accuracy: 0.4333
Epoch 3/10
162/162 [==============================] - 82s 507ms/step - loss: 0.9846 - accuracy: 0.5015 - val_loss: 1.0402 - val_accuracy: 0.4533
Epoch 4/10
162/162 [==============================] - 83s 510ms/step - loss: 0.8683 - accuracy: 0.5864 - val_loss: 0.8657 - val_accuracy: 0.5978
Epoch 5/10
162/162 [==============================] - 83s 511ms/step - loss: 0.7248 - accuracy: 0.6784 - val_loss: 0.9530 - val_accuracy: 0.5356
Epoch 6/10
162/162 [==============================] - 83s 515ms/step - loss: 0.5863 - accuracy: 0.7409 - val_loss: 0.9209 - val_accuracy: 0.5622
Epoch 7/10
162/162 [==============================] - 83s 510ms/step - loss: 0.4485 - accuracy: 0.8034 - val_loss: 0.8387 - val_ac

In [ ]:
model.save(Path().joinpath("Results", "model" + datetime.now().strftime('%b%d_%H-%M-%S')+ ".h5"))

In [ ]:
results = {}
for path in Path().joinpath(dataset_name, "test").glob("*.jpg"):
    image = Image.open(str(path)).convert("RGB")
    image = image.resize((img_w, img_h), Image.ANTIALIAS)
    image = np.array(image)
    image = np.expand_dims(image, 0)
    image = train_data_gen.normalize(image)
    image = tf.keras.applications.inception_v3.preprocess_input(image) # !!!!! Change this line with correct pre-processing (erase it if not transfer learning)
    results[path.name]= model.predict(image).argmax(axis=-1)[0] 
    
    print(str(path.name) + " " + str(results[path.name]))

In [ ]:
csv_fname = "results_" + datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'
Path().joinpath("Results").mkdir(parents=True, exist_ok=True)
with open(Path().joinpath("Results", csv_fname), "w") as f:
    f.write("Id,Category\n")
    for key, value in results.items():
        f.write(key + ',' + str(value) + '\n')